# Assignment 3

## AI usage

## Log

## Links 

- Github: https://github.com/Satheris/IND320_SMAA
- Streamlit app: https://ind320smaa-2eg32uba6uhmrknkwtxzar.streamlit.app/

## Coding 

### Imports 

In [1]:
import pandas as pd
import numpy as np

import openmeteo_requests
import requests_cache
from retry_requests import retry

### Download from openmeteo API

In [2]:
osloLat = 59.9127
osloLong = 10.7461

bergenLat = 60.393
bergenLong = 5.3242

trdLat = 63.4305
trdLong = 10.3951

tromsLat = 69.6489
tromsLong = 18.9551

kristLat = 58.1467
kristLong = 7.9956

citydict = {'priceArea': ['NO1', 'NO2', 'NO3', 'NO4', 'NO5'],
            'city': ['Oslo', 'Kristiansand', 'Bergen', 'Tromsø', 'Trondheim'],
            'longitude': [osloLong, kristLong, bergenLong, tromsLong, trdLong],
            'latitude': [osloLat, kristLat, bergenLat, tromsLat, trdLat]}


df = pd.DataFrame(citydict)

df

,priceArea,city,longitude,latitude
0,NO1,Oslo,10.7461,59.9127
1,NO2,Kristiansand,7.9956,58.1467
2,NO3,Bergen,5.3242,60.3930
3,NO4,Tromsø,18.9551,69.6489
4,NO5,Trondheim,10.3951,63.4305


In [10]:
# Function for reading openmeteo data 

def openmeteo_download(longitude, latitude, year):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": f"{year}-01-01",
        "end_date": f"{year}-12-31",
        "hourly": ["temperature_2m", "precipitation", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
        "models": "era5",
        "timezone": "Europe/Berlin",
        "wind_speed_unit": "ms",
    }
    responses = openmeteo.weather_api(url, params=params)

    return responses        

In [15]:
responses = openmeteo_download(bergenLong, bergenLat, 2019)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left",
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

Coordinates: 60.5°N 5.25°E
Elevation: 17.0 m asl
Timezone: b'Europe/Berlin'b'GMT+1'
Timezone difference to GMT+0: 3600s


,date,temperature_2m,wind_direction_10m,wind_speed_10m,wind_gusts_10m,precipitation
0,2018-12-31 23:00:00+00:00,6.70,0.4,11.853270,260.776154,22.700001
1,2019-01-01 00:00:00+00:00,6.55,0.5,13.322162,277.765076,24.400000
2,2019-01-01 01:00:00+00:00,6.80,0.9,13.505925,296.375275,22.299999
3,2019-01-01 02:00:00+00:00,6.85,0.7,14.621901,310.006195,23.700001
4,2019-01-01 03:00:00+00:00,6.55,0.6,15.487092,314.215271,27.400000


### Outliers and anomalies 

In [ ]:
starttime til datetime 

dt.tz_convert('Europe/Oslo')

merge(on='startTime')

response.Hourly

### Seasonal-Trend decomposition using LOESS (STL)

### Spectrogram

### Testing for Streamlit app